<a href="https://colab.research.google.com/github/DavoodSZ1993/Dive_into_Deep_Learning/blob/main/6_builder_guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 6.1 Layers and Modules

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.LazyLinear(256),   # input x 256
                    nn.ReLU(),            
                    nn.LazyLinear(10))    # 256 x 10

X = torch.rand(2, 20) # two samples (rows) & 20 features (columns)
X.shape, net(X).shape

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


(torch.Size([2, 20]), torch.Size([2, 10]))

### 6.1.1 A Custom Module

In [3]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()               # Performs necessary initialization from the parent class.
    self.hidden = nn.LazyLinear(256)
    self.out = nn.LazyLinear(10)

  def forward(self, X):              # Does the forward propagation for the given network.
    return self.out(F.relu(self.hidden(X)))

In [ ]:
net = MLP()
net(X).shape

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 10])

### 6.1.2 The Sequential Module

In [4]:
class MySequential(nn.Module):
  def __init__(self, *args):
    super().__init__()
    for idx, module in enumerate(args):
      self.add_module(str(idx), module)

  def forward(self, X):
    for module in self.children():
      X = module(X)
    return X

In [5]:
net_layers = (nn.LazyLinear(256),
              nn.ReLU(),
              nn.LazyLinear(10))
net = MySequential(*net_layers)

net(X).shape

torch.Size([2, 10])

### 6.1.3 Executing Code in the Forward Propagation Method

In [6]:
class FixedHiddenMLP(nn.Module):
  def __init__(self):
    super().__init__()

    self.rand_weight = torch.rand((20, 20))
    self.linear = nn.LazyLinear(20)

  def forward(self, X):
   
    X = self.linear(X)
    X = F.relu(X @ self.rand_weight + 1)
    X = self.linear(X)

    # Control flow
    while X.abs().sum() > 1:
      X /=2

    return X.sum()

In [ ]:
net = FixedHiddenMLP()
net(X)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor(0.0227, grad_fn=<SumBackward0>)

In [7]:
class NestMLP(nn.Module):
  def __init__(self):
    super().__init__()

    self.net = nn.Sequential(nn.LazyLinear(64),
                             nn.ReLU(),
                             nn.LazyLinear(32),
                             nn.ReLU())
    self.linear = nn.LazyLinear(16)

  def forward(self, X):
    return self.linear(self.net(X))

In [8]:
chimera = nn.Sequential(NestMLP(),
                        nn.LazyLinear(20),
                        FixedHiddenMLP())

chimera(X)

tensor(-0.0231, grad_fn=<SumBackward0>)

## 6.2 Parameter Management

In [9]:
import torch
from torch import nn

X = torch.rand(size=(2, 4))  # two inputs, four features

In [10]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

net(X).shape

torch.Size([2, 1])

### 6.2.1 Parameter Access

In [11]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.2177,  0.1978,  0.1426, -0.2947,  0.1307, -0.1862,  0.3150,  0.0651]])),
             ('bias', tensor([0.3267]))])

#### Targeted Parameters

In [12]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([0.3267]))

In [14]:
net[2].weight.grad == None

True

#### All Parameters at Once

In [13]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

### 6.2.2 Tied Parameters


In [15]:
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)

tensor([[0.2872],
        [0.2917]], grad_fn=<AddmmBackward0>)

In [16]:
print(net[2].weight.data[0] == net[4].weight.data[0])

net[2].weight.data[0, 0] = 100                        # Changes the weight in both layers.
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 6.3 Parameter Initialization

In [17]:
import torch
from torch import nn

X = torch.rand(size=(2, 4))

In [ ]:
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    nn.LazyLinear(1))

net(X).shape

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 1])

### 6.3.1 Built-in Initialization

In [18]:
def init_normal(module):
  if type(module) == nn.Linear:
    nn.init.normal_(module.weight, mean=0, std=0.01)
    nn.init.zeros_(module.bias)

In [19]:
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0109,  0.0024,  0.0188,  0.0155]), tensor(0.))

In [20]:
def init_constant(module):
  if type(module) == nn.Linear:
    nn.init.constant_(module.weight, 1)
    nn.init.zeros_(module.bias)

In [21]:
net.apply(init_constant)

net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [22]:
def init_xavier(module):
  if type(module) == nn.Linear:
    nn.init.xavier_uniform_(module.weight)

def init_42(module):
  if type(module) == nn.Linear:
    nn.init.constant_(module.weight, 42)

In [23]:
net[0].apply(init_xavier)
net[2].apply(init_42)

print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5194,  0.3576,  0.2914, -0.3258])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42., 42., 42., 42.]])


#### Custom Initialization

In [24]:
def my_init(module):
  if type(module) == nn.Linear:
    print("Init", *[(name, param.shape)
                      for name, param in module.named_parameters()][0])
    nn.init.uniform_(module.weight, -10, 10)
    module.weight.data *= module.weight.data >= 5

In [25]:
net.apply(my_init)

net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([8, 8])
Init weight torch.Size([1, 8])


tensor([[0.0000, -0.0000, -0.0000, 0.0000],
        [-0.0000, -0.0000, 0.0000, 6.6882]], grad_fn=<SliceBackward0>)

In [26]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.,  1.,  1.,  1.])

## 6.4 Lazy Initialization

In [27]:
!pip install d2l==1.0.0-alpha1.post0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.5 MB/s eta 0:00:00


In [28]:
import torch
from torch import nn
from d2l import torch as d2l

In [29]:
net = nn.Sequential(nn.LazyLinear(256),
                    nn.ReLU(),
                    nn.LazyLinear(10))

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [30]:
net[0].weight

<UninitializedParameter>

In [31]:
X = torch.rand(size=(2, 20))
net(X)
net[0].weight.shape

torch.Size([256, 20])

In [32]:
@d2l.add_to_class(d2l.Module)
def apply_init(self, inputs, init=None):
  self.forward(*inputs)
  if init is not None:
    self.net.apply(init)

## 6.5 Custom Layers

### 6.5.1 Layers without Parameters

In [33]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [34]:
class CenteredLayer(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, X):
    return X - X.mean()

In [35]:
layer = CenteredLayer()

X = torch.tensor([1.0, 2, 3, 4, 5])
layer(X)

tensor([-2., -1.,  0.,  1.,  2.])

In [36]:
net = nn.Sequential(nn.LazyLinear(128),
                    CenteredLayer())


In [37]:
Y = net(torch.rand(size=(4, 8)))
Y.mean()                          # Should be zero!!

tensor(-3.2596e-09, grad_fn=<MeanBackward0>)

### 6.5.2 Layers with Parameters

In [38]:
class MyLinear(nn.Module):
  def __init__(self, in_units, units):
    super().__init__()

    self.weight = nn.Parameter(torch.randn(in_units, units))
    self.bias = nn.Parameter(torch.randn(units,))

  def forward(self, X):
    linear = torch.matmul(X, self.weight.data) + self.bias.data
    return F.relu(linear)


In [39]:
linear = MyLinear(5, 3)
linear.weight, linear.bias

(Parameter containing:
 tensor([[-0.2309, -2.0704, -0.1101],
         [ 2.3534,  0.4064,  0.5021],
         [ 0.0216,  1.2691,  0.1689],
         [ 0.8855, -0.3487,  0.0986],
         [ 1.8550, -1.5288,  0.9273]], requires_grad=True),
 Parameter containing:
 tensor([-2.0492,  1.4929,  0.5352], requires_grad=True))

In [40]:
linear(torch.rand(2, 5))

tensor([[0.5775, 0.0000, 1.5027],
        [0.9836, 0.0000, 1.3411]])

In [41]:
net = nn.Sequential(MyLinear(64, 8),
                    MyLinear(8, 1))

net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

## 6.6 File I/O

### 6.6.1 Loading and Saving Tensors

In [42]:
import torch
from torch import nn
from torch.nn import functional as F

In [43]:
x = torch.arange(4)
torch.save(x, 'x-file')

In [44]:
x2 = torch.load('x-file')
x2 == x

tensor([True, True, True, True])

In [45]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [46]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')

mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### 6.6.2 Loading and Saving Model Parameters

In [47]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()

    self.hidden = nn.LazyLinear(256)
    self.output = nn.LazyLinear(10)

  def forward(self, X):
    return self.output(F.relu(self.hidden(X)))

In [48]:
net = MLP()

X = torch.randn(size=(2, 20))
Y = net(X)

In [49]:
torch.save(net.state_dict(), 'mlp.params')

In [50]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()                                      # evaluation mode

MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

In [51]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## 6.7 GPUs

### 6.7.1 Computing Devices

In [ ]:
import torch
from torch import nn
from d2l import torch as d2l

In [52]:
def cpu():
  return torch.device('cpu') # Returns cpu

def gpu(i=0):
  return torch.device(f'cuda:{i}')  # Returns cuda: 0 

cpu(), gpu(), gpu(1)       # cpu, gpu, cpu(because only one gpu is available on this cloud)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1))

In [54]:
def num_gpus():
  return torch.cuda.device_count()

num_gpus()

1

In [56]:
def try_gpu(i=0):
  if num_gpus() >= i + 1:
    return gpu(i)
  return cpu()

def try_all_gpus():
  return [gpu(i) for i in range(num_gpus())]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

### 6.7.2 Tensors and GPUs

In [57]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

#### Storage on the GPU

In [59]:
X = torch.ones(2, 3, device = try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [61]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y, Y.device

(tensor([[0.4282, 0.8906, 0.6433],
         [0.7156, 0.2573, 0.0744]]), device(type='cpu'))

#### Copying

In [62]:
Z = Y.cuda(0)
print(Y.device)
print(Z.device)

cpu
cuda:0


In [63]:
Z + X

tensor([[1.4282, 1.8906, 1.6433],
        [1.7156, 1.2573, 1.0744]], device='cuda:0')

In [64]:
Z.cuda(0) is Z

True

### 6.7.3 Neural Networks and GPUs

In [65]:
net = nn.Sequential(nn.LazyLinear(1))
net = net.to(device=try_gpu())

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [66]:
net(X)

tensor([[-0.0998],
        [-0.0998]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [67]:
net[0].weight.data.device

device(type='cuda', index=0)

In [68]:
@d2l.add_to_class(d2l.Trainer)
def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):
  self.save_hyperparameters()
  self.gpus = [d2l.gpu(i) for i in range(min(num_gpus, d2l.num_gpus()))]

@d2l.add_to_class(d2l.Trainer)
def prepare_batch(self, batch):                                                 # Moves each batch into GPU
  if self.gpus:
    batch = [a.to(self.gpus[0]) for a in batch]
  return batch

@d2l.add_to_class(d2l.Trainer)
def prepare_model(self, model):                                                 # Moves the model into GPU
  model.trainer = self
  model.board.xlim = [0, self.max_epochs]

  if self.gpus:
    model.to(self.gpus[0])
  self.model = model